In [9]:
import refinitiv.data as rd
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import cufflinks as cf
from scipy.stats import norm
cf.go_offline()

rd.open_session()

In [ ]:
def get_data(instrument, confidence_interval=0.8):
    _sd = datetime.now()
    _sd -= timedelta(days=730)
    df = rd.get_history(instrument, 'TRDPRC_1', start=_sd,
                        interval='daily', count=10000)
    # calculate log returns
    df_log = pd.DataFrame()
    df_log['TRDPRC_1'] = np.log(df['TRDPRC_1']) - \
        np.log(df['TRDPRC_1'].shift(1))
    df_log = df_log.dropna(axis=0)
    df_stats = df_log.describe()
    std = df_stats['TRDPRC_1']['std']
    mu = df_stats['TRDPRC_1']['mean']
    date_array = []
    vol_array = []
    lim_min = []
    lim_max = []
    lim_avg = []
    for t in range(90):
        _dt = datetime.timestamp(df.index[-1])
        _dt = datetime.fromtimestamp(_dt)
        date_array.append(_dt + timedelta(days=t))
        lim_max.append(df['TRDPRC_1'][-1] * np.exp(mu*t +
                       np.sqrt(t)*std*norm.ppf(confidence_interval)))
        lim_min.append(df['TRDPRC_1'][-1]/np.exp(-(mu*t -
                       np.sqrt(t)*std*norm.ppf(confidence_interval))))
        lim_avg.append(df['TRDPRC_1'][-1]*np.exp(mu*t))

    forecast = pd.DataFrame()
    forecast['UPPER LIM'] = lim_max
    forecast['LOWER LIM'] = lim_min
    forecast['TRDPRC_1'] = lim_avg
    forecast.index = date_array
    _df = df
    _df['UPPER LIM'] = np.nan
    _df['LOWER LIM'] = np.nan
    result = pd.concat([_df, forecast], sort=True)
    result.iplot(theme='solar', colors=[
                 '#6978F7', '#A325E9', '#96E05D', '#4A7FB9', '#E75A2D', '#FBE55A', '#8C8C8D', '#5A54F6'])
    return

In [ ]:
rd.close_session()